In [1]:
import warnings

import numpy as np
import pandas as pd

import cv2
from PIL import Image
from matplotlib import pyplot as plt 
import os
import sys
import warnings

import torch
import torch.nn as nn
from sklearn.metrics import precision_score, f1_score, recall_score, confusion_matrix

import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

import torchvision
import torchvision.transforms as transforms

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
warnings.filterwarnings("ignore")

## Dataset

In [2]:
class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img = np.array(self.images[idx])
        img = Image.fromarray(img)

        if self.transform:
            img = self.transform(img)

        label = torch.tensor(self.labels[idx]).type(torch.long)
        sample = (img, label)

        return sample

In [3]:
def load_data(path='fer2013.csv'):
    fer2013 = pd.read_csv(path)
    emotion_mapping = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}

    return fer2013, emotion_mapping

def ensure_color(image):
    if len(image.shape) == 2:
        return np.dstack([image] * 3)
    elif image.shape[2] == 1:
        return np.dstack([image] * 3)
    return image


def ensure_gray(image):
    try:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    except cv2.error:
        pass
    return image

In [5]:
def prepare_data():
    image_array = []
    image_label = []

    emotion_mapping = {'angry' : 0, 'disgust' : 1,'fear' : 2,'happy' : 3,'sad' : 4,'surprise' : 5, 'neutral' : 6}
    test_root_path = '../dataset/test/'
    for emotion_dir in os.listdir(test_root_path):
        for img_name in os.listdir(os.path.join(test_root_path,emotion_dir)):
            img = cv2.imread(os.path.join(test_root_path,emotion_dir,img_name))
            assert isinstance(img, np.ndarray)
            img = ensure_color(img)
            img = cv2.resize(img, (224,224))
            # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            image_array.append(img)
            image_label.append(emotion_dir)
    


    image_label = list(map(lambda x: emotion_mapping[x], image_label))

    return np.array(image_array), np.array(image_label)


In [6]:
def get_dataloaders(path='datasets/fer2013/fer2013.csv', bs=64, augment=True):
    """ Prepare train, val, & test dataloaders
        Augment training data using:
            - cropping
            - shifting (vertical/horizental)
            - horizental flipping
            - rotation

        input: path to fer2013 csv file
        output: (Dataloader, Dataloader, Dataloader) """



    xtest, ytest = prepare_data()
    transform = transforms.Compose(
    transforms=[ transforms.ToTensor()]
)

    # X = np.vstack((xtrain, xval))
    # Y = np.hstack((ytrain, yval))
    test = CustomDataset(xtest, ytest,transform)
    testloader = DataLoader(test, batch_size=2, shuffle=True)

    return testloader


In [7]:
for data in get_dataloaders():
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        print(labels.shape)
        print(inputs.shape)
        print(labels)
        break

torch.Size([64])
torch.Size([64, 3, 224, 224])
tensor([6, 4, 6, 5, 3, 3, 4, 5, 4, 3, 3, 3, 2, 3, 6, 2, 5, 1, 3, 5, 5, 2, 4, 2,
        3, 6, 0, 0, 6, 2, 3, 3, 2, 4, 5, 4, 6, 3, 5, 0, 3, 5, 4, 3, 5, 2, 6, 5,
        6, 3, 3, 4, 0, 5, 2, 3, 6, 3, 4, 4, 2, 4, 5, 0])


## Model Architecture

In [ ]:
# Load Trained Model
checkpoint = torch.load('./weights/VGGNet',map_location=torch.device('cpu'))
net = Vgg().to(device)
net.load_state_dict(checkpoint["params"])
net.eval()


Vgg(
  (conv1a): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1b): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2a): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2b): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3a): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3b): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4a): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4b): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (bn1a): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn1b): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2a): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn

#Evalutae model

In [9]:
def correct_count(output, target, topk=(1,)):
    """Computes the top k corrrect count for the specified values of k"""
    maxk = max(topk)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].contiguous().view(-1).float().sum(0, keepdim=True)
        res.append(correct_k)
    return res


def evaluate(net, dataloader, loss_fn, Ncrop, device):
    net = net.eval()
    loss_tr, n_samples = 0.0, 0.0

    y_pred = []
    y_gt = []

    correct_count1 = 0
    correct_count2 = 0

    for data in dataloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        if Ncrop:
            # fuse crops and batchsize
            bs, ncrops, c, h, w = inputs.shape
            inputs = inputs.view(-1, c, h, w)

            # forward
            outputs = net(inputs)

            # combine results across the crops
            outputs = outputs.view(bs, ncrops, -1)
            outputs = torch.sum(outputs, dim=1) / ncrops
        else:
            outputs = net(inputs)

        loss = loss_fn(outputs, labels)

        # calculate performance metrics
        loss_tr += loss.item()

        # accuracy
        counts = correct_count(outputs, labels, topk=(1, 2))
        correct_count1 += counts[0].item()
        correct_count2 += counts[1].item()

        _, preds = torch.max(outputs.data, 1)
        n_samples += labels.size(0)

        y_pred.extend(pred.item() for pred in preds)
        y_gt.extend(y.item() for y in labels)

    acc1 = 100 * correct_count1 / n_samples
    acc2 = 100 * correct_count2 / n_samples
    loss = loss_tr / n_samples
    print("--------------------------------------------------------")
    print("Top 1 Accuracy: %2.6f %%" % acc1)
    print("Top 2 Accuracy: %2.6f %%" % acc2)
    print("Loss: %2.6f" % loss)
    print("Precision: %2.6f" % precision_score(y_gt, y_pred, average='micro'))
    print("Recall: %2.6f" % recall_score(y_gt, y_pred, average='micro'))
    print("F1 Score: %2.6f" % f1_score(y_gt, y_pred, average='micro'))
    print("Confusion Matrix:\n", confusion_matrix(y_gt, y_pred), '\n')


In [10]:
from rmn import RMN, get_emo_model
net = get_emo_model()

In [11]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
criterion = nn.CrossEntropyLoss()
# Get data with no augmentation
testloader = get_dataloaders(augment=False)
print("Test")
evaluate(net, testloader, criterion, False, device)


Test
--------------------------------------------------------
Top 1 Accuracy: 50.626916 %
Top 2 Accuracy: 70.130956 %
Loss: 0.881979
Precision: 0.506269
Recall: 0.506269
F1 Score: 0.506269
Confusion Matrix:
 [[ 614   53   35   16   93   76   71]
 [  55   38    0    3   11    1    3]
 [ 322   17  130   37  241  172  105]
 [ 114   21   31 1255   50  248   55]
 [ 353   27   38   47  503   93  186]
 [ 105    4  123   23   15  545   16]
 [ 244   12   24   96  165  143  549]] 

